#### Steps

- import the CSV file into pandas dataframe
- clean the table name and remove all symbols, spaces, capital letters
- clean the column headers and remove all symbols, spaces, capital letters
- write the create table SQL statement
- import the data into the DB

#### Capabilites
- handle multiple files
- completely automatic (no code editing neccesary)

In [1]:
# import libraries
import os
import numpy as np
import pandas as pd
import psycopg2

In [24]:
#find CSV files in current directory
#pick only CSV files


os.chdir(r'C:/Users/user/Desktop/SampleData/datasets/')

csv_files = []
for file in os.listdir(os.getcwd()):
    if file.endswith('.csv'):
        csv_files.append(file)



In [28]:
#make new dir
dataset_dir = 'datasets'

#create bash command for new dir
#  mkdir dataset_dir

try:
    mkdir = 'mkdir {0}'.format(dataset_dir)
    os.system(mkdir)
except:
    pass

In [22]:
#move CSV files into the new dir

for csv in csv_files:
    
    mv_file ="mv '{0}' {1}".format(csv, dataset_dir)
    os.system(mv_file)
    print(mv_file)


 mv 'Customer Contracts$.csv' datasets
 mv 'Customer Demo.csv' datasets
 mv 'Customer Engagements.csv' datasets
 mv 'DATA$@SAMPLE.csv' datasets
 mv 'data_sample.csv' datasets


In [29]:
#create dataframe for each CSV file
data_path = os.getcwd()+'/'+dataset_dir+'/'

df = {}
for file in csv_files:
    try:
        df[file] = pd.read_csv(data_path+file)
    except UnicodeDecodeError:
        df[file] = pd.read_csv(data_path+file, encoding="ISO-8859-1")
    print(file)

Customer Contracts$.csv
Customer Demo.csv
Customer Engagements.csv
DATA$@SAMPLE.csv


In [42]:
#clean table & column names
# have only lower case letters
# remove all symbols
# replace space, -, /, @ with _
for k in csv_files:
    
    dataframe = df[k]
    
    clean_table_name = k.lower().replace(" ","_").replace("?","") \
                       .replace("-","_").replace(r"/","_").replace("\\","_") \
                       .replace("$","").replace("@","_").replace(r"(","") \
                       .replace(r")","").replace("%","")
    
    
    #remove .csv ext
    tbl_name = '{0}'.format(clean_table_name.split('.')[0])
    
    
    dataframe.columns = [x.lower().replace(" ","_").replace("?","") \
                       .replace("-","_").replace(r"/","_").replace("\\","_") \
                       .replace("$","").replace("@","_").replace(r"(","") \
                       .replace(r")","").replace("%","") for x in dataframe.columns]
    
    
    #replacement dictionary that maps pandas dtypes to sql dtypes
    dt_replacements = {
    'object' : 'varchar' ,
    'int64' : 'int' ,
    'float64' : 'float' ,
    'datetime64' : 'timestamp' ,
    'timedelta64[ns]' : 'varchar'
    }
    
    #table schema
    col_str = ", ".join("{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(dt_replacements)))
    
    #open connection to DB
    host = 'ronamazoninstance.c9n0aimuu7iu.eu-central-1.rds.amazonaws.com'
    dbname = 'rondb'
    user = 'ronadmin'
    password = 'Korki123'

    conn_string = "host=%s dbname=%s user=%s password=%s" % (host, dbname, user, password)
    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()               
    print('opened db')
    
    #drop potential duplicate tables
    cursor.execute("drop table if exists %s;" % (tbl_name))
    
    #create the table
    cursor.execute("create table %s (%s);" % (tbl_name, col_str))
    print('{0} was created'.format(tbl_name))
    
    #insert values into table

    #save df to csv
    dataframe.to_csv(k, header=dataframe.columns, index=False, encoding='utf-8')

    #open the csv file, save as an object
    my_file = open(k)
    print('file is open in memory')
    
    #upload the file and inesrt into the DB

    SQL_STATEMENT = """
    COPY %s FROM STDIN WITH
     CSV
     HEADER
     DELIMITER AS ','
    """

    cursor.copy_expert(sql=SQL_STATEMENT % tbl_name, file=my_file)
    print('file copied to db')
    
    #give permissions on table to anyone and close the connection
    cursor.execute("grant select on table %s to public" % tbl_name)
    conn.commit()
    cursor.close()
    print('table {0} import to db is completed'.format(tbl_name))
    
#for end message
print('all tables have been imported into the DB.')

opened db
customer_contracts was created
file is open in memory
file copied to db
table customer_contracts import to db is completed
opened db
customer_demo was created
file is open in memory
file copied to db
table customer_demo import to db is completed
opened db
customer_engagements was created
file is open in memory
file copied to db
table customer_engagements import to db is completed
opened db
data_sample was created
file is open in memory
file copied to db
table data_sample import to db is completed
all tables have been imported into the DB.
